## How to use Desk
Desk is Franka Emika‘s web-based, intuitive and graphical programming and user interface.
* link: https://{PANDA-IP}/desk/

```Pilot mode``` [Tablet / End-effector]

* Tablet: control by apps on GUI
        drag & drop apps icon on blank area and save position, set velocity & acceleration of motion
        click 'Run the task' button to run teached motion 
* End-effector: control by buttons which located on top of the robot


```Guiding mode``` [Translation / Rotation / Free move / User]

* Translation: Only translatioal movement is possible on end-effector
* Rotation: Only rotational movement is possible on end-effector
* Rotation: Both rotational and translatioal movements are possible on end-effector
* User: User can restrict direction of movement on end-effector(X, Y, Z, R, P, Y)



```Joints``` [Lock / Unlock]

* Lock: lock all joints mechanically to make Panda cannot move 
* Unlock: Unlock the joints to make Panda can move

```Robot status```

* status of each part of Panda
* the status represented by color which is refered to below 'Operating mode'

## Operating mode
```WHITE [Interactive]```
* robot can move interacitively (direct teaching; push enabling button)

```BLUE [Activation]```

* recieve signal from PC and move based on the signal (pull enabling button)

```GREEN [Automatic execution]```
* carrying out an automatic program (moving from Desk)

```YELLOW [Lock]```
* joints are locked mechanically, so it cannot move (immediately after turn on Panda)

```PINK [Conflict]```

* occured conflict from get different operating mode signals simultaneously  

```RED [Error]```

* occured some errors

## How to implement 
```Controller (joint_control_rnb.launch)```
* see below cell

```Web UI```
* After executing joint_control_rnb.launch, control WebUI is on http://{PANDA-IP}:9990/
```
Select controller : change a controller
Set Default : set current controller as default
Apply : after change gains, press 'Apply' button
Save & Load : save current gain values & load saved values
Restart log & Pause log : start & pause log
Download full : save logged data as .csv file (only last 30 secs data can be saved)
```

```
Tip : before download saved data, pause log at the first
```
```Making trajectory (trajectroy_client)```
* follow this 'trajectory_client_example'

## Launch controller
* Execute below command after replacing {PANDA-IP} with the IP of Panda robot
```bash
roslaunch panda_control joint_control_rnb.launch robot_ip:={PANDA-IP} load_gripper:=false
```

* Tip
```
If an interruption (ex. error) occured on controller, push & pull enabling button and restart controller
```

In [1]:
from utils.trajectory_client.trajectory_client import *

## Initialize default client (no gripper function)

In [ ]:
traj_client = TrajectoryClient("localhost")

### If you are using panda, run below cell to use panda gripper

In [ ]:
from panda_trajectory_client import PandaTrajectoryClient
traj_client = PandaTrajectoryClient(server_ip="192.168.1.4", robot_ip="192.168.1.13")

### If you are using Indy7, run below cell

In [ ]:
from indy_trajectory_client import IndyTrajectoryClient
traj_client = IndyTrajectoryClient(server_ip="192.168.21.6")

## Testing short motion

In [ ]:
Q1 = traj_client.get_qcur()
Q2 = np.add(Q1, np.pi/32)

In [ ]:
traj_client.move_joint_s_curve(Q1, N_div=100)

In [ ]:
traj_client.move_joint_s_curve(Q2, N_div=100)

## direct teaching

In [ ]:
qlist=[]

#### Set the controller to gravity compensator and run below cell multiple times to save waypoints
* You don't have to press enabling button for direct teaching

In [ ]:
qlist.append(traj_client.get_qcur())
print("q: {}".format(qlist[-1]))


#### list all saved waypoints

In [ ]:
for i_q, q in enumerate(qlist):
    print("q{}: {}".format(i_q, q))

#### update target position and reset controller for safety

In [ ]:
traj_client.move_joint_s_curve(traj_client.get_qcur(), N_div=50)
traj_client.reset()

#### Set controller to NRIC_PD or other active one  - if the robot moves abruptly, restart the controller program

#### Run below cell to repeat the waypoint motion (move_joint_s_curve version)

In [ ]:
for _ in range(3):
    for q in qlist:
        traj_client.move_joint_s_curve(q, N_div=50) # adjust speed by N_div

#### Run below cell to repeat the waypoint motion (move_joint_wp version)

In [ ]:
for _ in range(3):
    traj_client.move_joint_wp(qlist, vel_lims=0.5, acc_lims=0.5, auto_stop=True) # adjust speed by vel_lims and acc_lims

### using gripper

In [ ]:
traj_client.grasp(True)

In [ ]:
traj_client.grasp(False)

## Task trajectory generation with indy

#### Prepare urdf content

In [ ]:
from urdf_parser_py.urdf import URDF
from joint_utils import *

urdf_content = URDF.from_xml_file("urdf/indy7.urdf")
link_names = get_link_names(urdf_content)
joint_names = get_joint_names(urdf_content)
base_link, tip_link = link_names[0], link_names[-1]

#### calculating transformation matrix

In [ ]:
Tbe = get_tf(tip_link, list2dict([0]*6, joint_names), urdf_content, from_link=base_link)
print(np.round(Tbe, 3))

#### calculating line sweep trajectory

In [ ]:
Q0 = traj_client.get_qcur()
dP = [0.1,0,0]
traj, succ = get_sweep_traj(urdf_content, joint_names, tip_link, dP, Q0)

#### following generated trajectory 

In [ ]:
traj_tot, traj_time = traj_client.move_joint_wp(traj, vel_lims=0.2, acc_lims=0.2, auto_stop=True)